# **Loss Functions in a Deterministic Optimization Setting**

First, we import both the project modules and the necessary libraries:

In [2]:
from src.ts_simulator.distributions import NormalDist, CauchyDist, GumbelDist, LogNormalDist, ParetoDist
from src.ts_simulator.distributions import IDistSimulator
from src.ts_simulator.simulator import TimeSeriesSimulator
from src.graphics.classes import PlotSimulatedTS
from src.dnn.utils import train_and_predict, l_infinity_loss, quantile_loss_fn

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from typing import Dict
import re
import keras

### Time Series Simulation

Fix number of points and random seed

In [4]:
n: int = 280
seed: int = 12
d: int = 300

Create simulated time series with a specific functional form

In [5]:
x: np.ndarray = np.linspace(0, 1, n)
sin_ts: np.ndarray = np.sin(12 * x + 1)

Instantiate utils for series:

In [6]:
simul = TimeSeriesSimulator()
plotter = PlotSimulatedTS()

output_dir = os.path.join(
    os.getcwd(), "experiments/sim_ts_determ/ts_plots")
os.makedirs(output_dir, exist_ok=True)


Define error distributions for determinisitic time series (to add noise). In this case, we use parameters such that the differences in the specific quantities we are studying are obvious.

In [7]:
normal: IDistSimulator = NormalDist({"loc": 0, "scale": 0.25})
cauchy: IDistSimulator = CauchyDist({"loc": 0, "scale": 0.01})
gumbel: IDistSimulator = GumbelDist({"loc": 0, "scale": 1.1})
lognormal: IDistSimulator = LogNormalDist({"loc": 0, "scale": 0.65})
pareto: IDistSimulator = ParetoDist({"xm": 0.01, "alpha": 1.25})

errors: Dict[str, IDistSimulator] = {
    "Normal": normal,
    "Cauchy": cauchy,
    "Gumbel": gumbel,
    "LogNormal": lognormal,
    "Pareto": pareto
}

mean_dist: Dict[str, float] = {
    "Normal": normal.theory()["mean"],
    "Cauchy": cauchy.theory()["mean"],
    "Gumbel": gumbel.theory()["mean"],
    "LogNormal": lognormal.theory()["mean"],
    "Pareto": pareto.theory()["mean"]
}

median_dist: Dict[str, float] = {
    "Normal": normal.theory()["median"],
    "Cauchy": cauchy.theory()["median"],
    "Gumbel": gumbel.theory()["median"],
    "LogNormal": lognormal.theory()["median"],
    "Pareto": pareto.theory()["median"]
}

q_1: Dict[str, float] = {
    "Normal": normal.quantile(0.1),
    "Cauchy": cauchy.quantile(0.1),
    "Gumbel": gumbel.quantile(0.1),
    "LogNormal": lognormal.quantile(0.1),
    "Pareto": pareto.quantile(0.1)
}

q_9: Dict[str, float] = {
    "Normal": normal.quantile(0.9),
    "Cauchy": cauchy.quantile(0.9),
    "Gumbel": gumbel.quantile(0.9),
    "LogNormal": lognormal.quantile(0.9),
    "Pareto": pareto.quantile(0.9)
}

We need now to simulate various time series and tile them together.

In [8]:
t_repeated: np.ndarray = np.tile(x, (d, 1))        # (d, n)
target: np.ndarray = np.tile(sin_ts, (d, 1))       # (d, n)

dict_series = {}

for name, dist in errors.items():
    noise = dist.draw(size=d*n).reshape(d, n)      # (d, n)
    data  = np.tile(sin_ts, (d, 1)) + noise        # (d, n)

    dict_series[name] = data                       ### NO aplastes a (-1,1)

    fig, ax = plt.subplots(figsize=(10, 6))
    for i in range(1, 21):
        ax.plot(x, data[i], color="gray", alpha=0.25)
    ax.plot(x, data[0], label='Noisy Signal', color="red")
    ax.plot(x, sin_ts + mean_dist[name], label='Deterministic Term',
            linestyle='--', color="black", alpha=0.7)
    
    y_min = min(data[0].min(), (sin_ts + mean_dist[name]).min())
    y_max = max(data[0].max(), (sin_ts + mean_dist[name]).max())
    ax.set_ylim(y_min, y_max)

    x_min, x_max = x.min(), x.max()
    ax.set_xlim(x_min, x_max)
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    ax.set_title(f"Time Series with {name} Noise")
    ax.legend()
    fig.savefig(os.path.join(output_dir, f"series_{name}.png"))
    plt.close(fig)

### Train ANNs

Define loss functions and ANN parameters

In [9]:
loss_functions = {
    "L_1": "mae",
    "L_2": "mse",
    "L_inf": l_infinity_loss,
    "Quantile_0.1": quantile_loss_fn(0.1),
    "Quantile_0.9": quantile_loss_fn(0.9),
}

seq_length = n
batch_size = 32
epochs = 250
learning_rate = 0.01

scaler = StandardScaler()  # or MinMaxScaler()

dict_predictions = {}
histories = {}

model = keras.models.Sequential([
        keras.layers.Input(shape=(seq_length,)),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(seq_length, activation='linear')
    ])
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

Train the different ANNs and obtain the predictions for each possible time series and each possible loss function

In [10]:
for series_name, series_data in dict_series.items():
    X = t_repeated
    y = series_data
    for loss_name, loss_fn in loss_functions.items():
        print(
            f"\n\nTraining {series_name} with {loss_name} loss function...\n\n")
        model.compile(optimizer=optimizer, loss=loss_fn)
        history = model.fit(x=X, y=y, batch_size=batch_size, epochs=epochs)
        preds = model.predict(X)

        dict_predictions[f"{series_name}_{loss_name}"] = preds[0]
        histories[f"{series_name}_{loss_name}"]  = history



Training Normal with L_1 loss function...


Epoch 1/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.4758  
Epoch 2/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2262 
Epoch 3/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2119 
Epoch 4/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2057 
Epoch 5/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2047 
Epoch 6/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2002 
Epoch 7/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2005 
Epoch 8/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2016 
Epoch 9/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2019 
Epoch 10/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2017 
Epoch 11/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2027 
Epoch 12/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2012 
Epoch 13/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2002 
Epoch 14/250
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2006 
Epoch 15/2

Create ANN learning curves for each time series with a specific noise:

In [11]:
base_dir = os.path.join(os.getcwd(), "experiments", "sim_ts_determ")
plot_dir = os.path.join(base_dir, "learning_curves")
os.makedirs(plot_dir, exist_ok=True)

def safe_name(s: str) -> str:
    return re.sub(r'[^A-Za-z0-9_.-]+', '-', str(s))

use_log_scale = True

for series_name in dict_series.keys():
    series_dir = os.path.join(plot_dir, safe_name(series_name))
    os.makedirs(series_dir, exist_ok=True)

    for loss_name in loss_functions.keys():
        key = f"{series_name}_{loss_name}"
        if key not in histories:
            alt_key = f"{series_name}_{loss_name.replace('_', '')}"
            if alt_key in histories:
                key = alt_key

        hist = histories[key].history
        train = hist.get("loss", None)
        val   = hist.get("val_loss", None)

        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(train, label=f"{loss_name} (train)")
        if val is not None:
            ax.plot(val, label=f"{loss_name} (val)", linestyle="--")

        if use_log_scale:
            ax.set_yscale("log")

        ax.set_title(f"Learning Curve — Dist='{series_name}' — Loss='{loss_name}'")
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Loss")
        ax.legend(fontsize="small")
        ax.grid(True)
        plt.tight_layout()

        fname = f"learning_curve_{safe_name(series_name)}_{safe_name(loss_name)}.png"
        out_path = os.path.join(series_dir, fname)
        fig.savefig(out_path, dpi=150)
        plt.close(fig)

We now look at the differences between L1 and L2:

In [12]:
base_dir = os.path.join(os.getcwd(), "experiments", "sim_ts_determ")
diff_plot_dir = os.path.join(base_dir, "difference_plots")
os.makedirs(diff_plot_dir, exist_ok=True)

for series_name in dict_series.keys():
    if f"{series_name}_L_1" in dict_predictions and f"{series_name}_L_2" in dict_predictions:
        l1_preds = dict_predictions[f"{series_name}_L_1"]
        l2_preds = dict_predictions[f"{series_name}_L_2"]
        diff_l1_l2 = l1_preds - l2_preds
        
        # Theoretical difference between median and mean
        theoretical_diff = median_dist[series_name] - mean_dist[series_name]
        mean_diff = np.mean(diff_l1_l2)

        plt.figure(figsize=(10, 6))
        plt.plot(x, diff_l1_l2, label="L1 - L2 Difference", color="purple")
        plt.axhline(y=theoretical_diff, color='red', linestyle='--', 
                   label=f'Theoretical Difference: {theoretical_diff:.4f}')
        plt.axhline(y=mean_diff, color='blue', linestyle='-.', 
                   label=f'Mean Difference: {mean_diff:.4f}')
        plt.title(f"Difference Between L1 and L2 Predictions — {series_name}")
        plt.xlabel("Time")
        plt.ylabel("Difference")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        fname = f"diff_L1_L2_{safe_name(series_name)}.png"
        out_path = os.path.join(diff_plot_dir, fname)
        plt.savefig(out_path, dpi=150)
        plt.close()

    if f"{series_name}_Quantile_0.9" in dict_predictions and f"{series_name}_Quantile_0.1" in dict_predictions:
        q9_preds = dict_predictions[f"{series_name}_Quantile_0.9"]
        q1_preds = dict_predictions[f"{series_name}_Quantile_0.1"]
        diff_q9_q1 = q9_preds - q1_preds
        
        # Theoretical difference between quantiles
        theoretical_diff = q_9[series_name] - q_1[series_name]
        mean_diff = np.mean(diff_q9_q1)

        plt.figure(figsize=(10, 6))
        plt.plot(x, diff_q9_q1, label="0.9 Quantile - 0.1 Quantile Difference", color="orange")
        plt.axhline(y=theoretical_diff, color='red', linestyle='--',
                   label=f'Theoretical Difference: {theoretical_diff:.4f}')
        plt.axhline(y=mean_diff, color='blue', linestyle='-.',
                   label=f'Mean Difference: {mean_diff:.4f}')
        plt.title(f"Difference Between 0.9 and 0.1 Quantile Predictions — {series_name}")
        plt.xlabel("Time")
        plt.ylabel("Difference")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        fname = f"diff_Quantiles_{safe_name(series_name)}.png"
        out_path = os.path.join(diff_plot_dir, fname)
        plt.savefig(out_path, dpi=150)
        plt.close()

Create prediction plots for time series with various noises (separetely):

In [13]:
base_dir = os.path.join(os.getcwd(), "experiments", "sim_ts_determ")
plot_dir = os.path.join(base_dir, "prediction_plots")
os.makedirs(plot_dir, exist_ok=True)

for key, pred_array in dict_predictions.items():
    # Clave esperada: f"{series_name}_{loss_name}"
    if "_" in key:
        series_name, loss_name = key.split("_", 1)
    else:
        series_name, loss_name = key, "unknown"

    if series_name == "Original":
        continue
    
    if loss_name == "L_1":
        quantity = np.repeat(median_dist[series_name],n)
    elif loss_name == "L_2":
        quantity = np.repeat(mean_dist[series_name],n)
    elif loss_name == "L_inf":
        quantity = np.repeat(((np.max(dict_series[series_name][0])+np.min(dict_series[series_name][0]))/2),n)
    elif loss_name == "Quantile_0.1":
        quantity = np.repeat(q_1[series_name],n)
    elif loss_name == "Quantile_0.9":
        quantity = np.repeat(q_9[series_name],n)

    series_dir = os.path.join(plot_dir, safe_name(series_name))
    os.makedirs(series_dir, exist_ok=True)

    plt.figure(figsize=(10, 6))
    plt.plot(
        x, sin_ts + mean_dist[series_name],
        label=f"True Signal",
        alpha=1, color="purple", linestyle=":"
    )
    plt.plot(
        x, sin_ts + quantity,
        label=f"Approximated Quantity",
        alpha=1, color="black", linestyle="--"
    )
    plt.plot(
        x, dict_series[series_name][0],
        label=f"Time Series with {series_name} Noise",
        alpha=0.35, color="red"
    )
    plt.plot(
        np.linspace(0,1,n), pred_array,
        label=f"Prediction ({loss_name})",
        linestyle='-', color="blue", alpha=0.65
    )
    plt.title(f"ANN Prediction — Noise: {series_name} | Loss: {loss_name}")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    fname = f"pred_{safe_name(series_name)}_{safe_name(loss_name)}.png"
    out_path = os.path.join(series_dir, fname)
    plt.savefig(out_path, dpi=150)
    plt.close()

Create grid error distributions histograms for the predictions

In [14]:
base_dir = os.path.join(os.getcwd(), "experiments", "sim_ts_determ")
hist_dir = os.path.join(base_dir, "error_histograms")
os.makedirs(hist_dir, exist_ok=True)

bins = 50
alpha = 0.5

def unique_preserve_order(xs):
    seen, out = set(), []
    for x in xs:
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out

for series_name, original in dict_series.items():
    errors = {}
    for key, pred_array in dict_predictions.items():
        s, loss_name = key.split("_", 1)
        if s != series_name:
            continue
        y_true = np.asarray(original[0]).ravel()
        y_pred = np.asarray(pred_array).ravel()
        m = min(len(y_true), len(y_pred))     # iguala longitudes por seguridad
        err = y_pred[:m] - y_true[:m]         # error t a t
        errors[loss_name] = err

    loss_names_all = list(errors.keys())
    loss_names_no_linf = [ln for ln in loss_names_all if ln != 'L_inf']
    preferred = []
    for tag in ('L1', 'L2'):
        for ln in loss_names_no_linf:
            if ln == tag:
                preferred.append(ln)
                break
    others = [ln for ln in loss_names_no_linf if ln not in preferred]
    ordered = unique_preserve_order(preferred + others)

    selected = ordered[:4]

    pairs = []
    if len(selected) >= 2:
        pairs.append((selected[0], selected[1]))
    if len(selected) >= 4:
        pairs.append((selected[2], selected[3]))
    ncols = len(pairs)
    fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=(20,6),
                             sharex=True, sharey=True)
    axes = np.atleast_1d(axes)

    for i, (ax, (ln1, ln2)) in enumerate(zip(axes, pairs)):
        color1 = f"C{2*i}"
        color2 = f"C{2*i+1}"

        ax.hist(errors[ln1], bins=bins, density=True, alpha=alpha, label=ln1, color=color1)
        ax.hist(errors[ln2], bins=bins, density=True, alpha=alpha, label=ln2, color=color2)

        mean1 = float(np.mean(errors[ln1]))
        mean2 = float(np.mean(errors[ln2]))
        ax.axvline(mean1, linestyle=':', linewidth=2, color=color1, label=f"{ln1} mean")
        ax.axvline(mean2, linestyle=':', linewidth=2, color=color2, label=f"{ln2} mean")

        ax.set_title(f"{series_name} — {ln1} vs {ln2}")
        ax.set_ylabel("Densidad")
        ax.legend(fontsize='small')
        ax.grid(True, linestyle='--', alpha=0.3)

    axes[-1].set_xlabel("Error (ŷₜ – yₜ)")

    plt.tight_layout()
    out_path = os.path.join(hist_dir, f"grid2_hist_errors_{series_name}.png")
    fig.savefig(out_path, dpi=150)
    plt.close(fig)

Prediction Grids:

In [ ]:
# =========================
# Grid de predicciones (sim_ts_determ): L1/L2 y Quantiles 0.1/0.9
# =========================
import os, re
import numpy as np
import matplotlib.pyplot as plt

base_dir = os.path.join(os.getcwd(), "experiments", "sim_ts_determ")
grid_pred_dir = os.path.join(base_dir, "grid_predictions")
os.makedirs(grid_pred_dir, exist_ok=True)

def safe_name(s: str) -> str:
    return re.sub(r'[^A-Za-z0-9_.-]+', '-', str(s))

def get_true_series(series_arr: np.ndarray) -> np.ndarray:
    """Ensure a 1D true series (we use the first realisation)."""
    s = np.asarray(series_arr)
    if s.ndim == 2:
        return s[0].ravel()
    return s.ravel()

def align_pred_axis(x_full: np.ndarray, y_true: np.ndarray, y_pred: np.ndarray):
    """
    Align axes for plotting:
    - If len(y_pred) == len(y_true): use x_full for both.
    - If len(y_pred) == len(y_true)-1: shift pred to x_full[1:].
    - Else: trim both to the minimum length.
    """
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    n_true, n_pred = y_true.size, y_pred.size

    if n_pred == n_true:
        return x_full, y_true, x_full, y_pred
    elif n_pred == max(n_true - 1, 0):
        return x_full, y_true, x_full[1:1+n_pred], y_pred
    else:
        m = min(n_true, n_pred)
        return x_full[:m], y_true[:m], x_full[:m], y_pred[:m]

def get_pred(series_name: str, suffix: str):
    """Return (found_flag, y_pred_vector) for key f'{series_name}_{suffix}'."""
    key = f"{series_name}_{suffix}"
    if key not in dict_predictions:
        return False, None
    return True, np.asarray(dict_predictions[key]).ravel()

# === FONDO: series grises ===
def _plot_background_gray(ax, series_arr: np.ndarray, x_axis: np.ndarray, max_lines: int = 20, alpha: float = 0.25):
    """
    Dibuja hasta `max_lines` realizaciones (excluyendo la 0, que usas como y_true)
    como fondo en gris con alpha dado.
    """
    s = np.asarray(series_arr)
    if s.ndim < 2:
        return
    d = s.shape[0]
    for i in range(1, min(max_lines + 1, d)):
        ax.plot(x_axis, s[i], color="gray", alpha=alpha, linewidth=1)

# Bucle principal
for series_name, series_arr in dict_series.items():
    # True series (first realisation) and x
    y_true = get_true_series(series_arr)  # shape (n,)
    if y_true.size == 0:
        continue

    # Grab predictions if present
    has_L2, yL2 = get_pred(series_name, "L_2")
    has_L1, yL1 = get_pred(series_name, "L_1")
    has_Q09, yQ09 = get_pred(series_name, "Quantile_0.9")
    has_Q01, yQ01 = get_pred(series_name, "Quantile_0.1")

    left_has_any  = has_L1 or has_L2
    right_has_any = has_Q09 or has_Q01
    if not (left_has_any or right_has_any):
        continue

    # Prepare output dir
    series_dir = os.path.join(grid_pred_dir, safe_name(series_name))
    os.makedirs(series_dir, exist_ok=True)

    # Figure
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6), sharey=True)

    # ----- Left panel: L1 vs L2 -----
    ax = axes[0]
    _plot_background_gray(ax, series_arr, x)  # <<< fondo gris
    if has_L2:
        _, _, x_pred_L2, y_pred_L2 = align_pred_axis(x, y_true, yL2)
        ax.plot(x_pred_L2, y_pred_L2, label="Prediction (L2)", color="red", alpha=0.9)
    if has_L1:
        _, _, x_pred_L1, y_pred_L1 = align_pred_axis(x, y_true, yL1)
        ax.plot(x_pred_L1, y_pred_L1, label="Prediction (L1)", color="blue", alpha=0.9)
    ax.set_title(f"Predictions — {series_name} (L1 vs L2)")
    ax.set_xlabel("t")
    ax.set_ylabel("Value")
    ax.legend(fontsize="small")
    ax.grid(True, linestyle="--", alpha=0.3)

    # Cap axes
    ax.set_xlim(x.min(), x.max())
    y_min, y_max = y_true.min(), y_true.max()
    ax.set_ylim(y_min, y_max)

    # ----- Right panel: quantiles 0.1 vs 0.9 -----
    ax = axes[1]
    _plot_background_gray(ax, series_arr, x)  # <<< fondo gris
    if has_Q09:
        _, _, x_pred_Q09, y_pred_Q09 = align_pred_axis(x, y_true, yQ09)
        ax.plot(x_pred_Q09, y_pred_Q09, label="Prediction (Quantile 0.9)", color="orange", alpha=0.9)
    if has_Q01:
        _, _, x_pred_Q01, y_pred_Q01 = align_pred_axis(x, y_true, yQ01)
        ax.plot(x_pred_Q01, y_pred_Q01, label="Prediction (Quantile 0.1)", color="purple", alpha=0.9)
    ax.set_title(f"Predictions — {series_name} (Quantiles)")
    ax.set_xlabel("t")
    ax.legend(fontsize="small")
    ax.grid(True, linestyle="--", alpha=0.3)

    # Same axis capping
    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(y_min, y_max)

    plt.tight_layout()
    fname = f"grid_predictions_{safe_name(series_name)}.png"
    out_path = os.path.join(series_dir, fname)
    fig.savefig(out_path, dpi=150)
    plt.close(fig)

print(f"Saved grid figures under: {grid_pred_dir}")


Saved grid figures under: c:\Users\Iker\Desktop\Algoritmos_Perrotes\ModellingError_TS\experiments\sim_ts_determ\grid_predictions
